# Initial EDA for Washington Databreach Dataframe

## Imports & Initial
***

### Imports

In [4]:
# autoreload
%load_ext autoreload
%autoreload 2

In [6]:
# basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [5]:
# pandas formatting
pd.options.display.float_format = '{:.0f}'.format

### Read csv and turn into pandas dataframe.

In [10]:
databreach_df = pd.read_csv('../data/WA_Databreach_20231102.csv')
databreach_df.head()

,DateAware,DateSubmitted,DataBreachCause,DateStart,DateEnd,Name,Id,CyberattackType,WashingtoniansAffected,IndustryType,...,BreachLifecycleRange,DaysToContainBreach,DaysToIdentifyBreach,DaysBreachLifecycle,DiscoveredInProgress,DaysOfExposure,DaysElapsedBetweenEndAndDiscovery,EndedOnDayDiscovered,DaysElapsedBeforeNotification,DaysOfExposureRange
0,09/20/2017 12:00:00 AM,11/15/2017 12:00:00 AM,Theft or Mistake,09/20/2017 12:00:00 AM,09/20/2017 12:00:00 AM,Western State Hospital,9750,NaN,515,Health,...,0,0,0,0,True,0,NaN,True,56,0
1,11/15/2016 12:00:00 AM,11/21/2017 12:00:00 AM,Cyberattack,10/13/2016 12:00:00 AM,11/15/2016 12:00:00 AM,"Uber Technologies, Inc.",9752,NaN,10888,Business,...,1-99,0,33,33,True,33,NaN,True,371,1-99
2,NaN,11/27/2017 12:00:00 AM,Cyberattack,10/26/2016 12:00:00 AM,10/19/2017 12:00:00 AM,"Bulletproof 360, Inc.",9753,Malware,4077,Business,...,Unknown,NaN,NaN,NaN,False,358,NaN,False,NaN,300-399
3,09/06/2017 12:00:00 AM,12/01/2017 12:00:00 AM,Unauthorized Access,06/15/2016 12:00:00 AM,11/06/2017 12:00:00 AM,JAM Paper & Envelope,9754,NaN,590,Business,...,500+,61,448,509,True,509,NaN,False,86,500+
4,10/06/2017 12:00:00 AM,12/11/2017 12:00:00 AM,Cyberattack,07/01/2015 12:00:00 AM,10/06/2017 12:00:00 AM,"Combat Brands, LLC",9755,Malware,956,Business,...,500+,0,828,828,True,828,NaN,True,66,500+


### Get Information

In [11]:
databreach_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   DateAware                          966 non-null    object 
 1   DateSubmitted                      990 non-null    object 
 2   DataBreachCause                    990 non-null    object 
 3   DateStart                          862 non-null    object 
 4   DateEnd                            804 non-null    object 
 5   Name                               990 non-null    object 
 6   Id                                 990 non-null    int64  
 7   CyberattackType                    649 non-null    object 
 8   WashingtoniansAffected             958 non-null    float64
 9   IndustryType                       990 non-null    object 
 10  BusinessType                       467 non-null    object 
 11  Year                               990 non-null    int64  

### Clean Dataframe

Steps:
1. Grab only necessary columns
2. Change DatabreachCause:Cyberattack, CyberattackType:NaN to DatabreachCause:Cyberattack, CyberattackType:Unreported

In [16]:
# Dataframe Cleaning

# Step 1
cleandf = databreach_df[['DataBreachCause', 'CyberattackType', 'WashingtoniansAffected', 'IndustryType', 'BusinessType', 'Year', 'DaysOfExposure', 'DiscoveredInProgress', 'EndedOnDayDiscovered']]
# Step 2
cleandf.loc[:, 'CyberattackType'][(cleandf['DataBreachCause'] == 'Cyberattack') 
                                                & (cleandf['CyberattackType'].isnull())] = 'Unreported'


cleandf

C:\Users\isaac\AppData\Local\Temp\ipykernel_30592\1099039990.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf.loc[:, 'CyberattackType'][(cleandf['DataBreachCause'] == 'Cyberattack')


,DataBreachCause,CyberattackType,WashingtoniansAffected,IndustryType,BusinessType,Year,DaysOfExposure,DiscoveredInProgress,EndedOnDayDiscovered
0,Theft or Mistake,NaN,515,Health,NaN,2018,0,True,True
1,Cyberattack,Unreported,10888,Business,Transportation,2018,33,True,True
2,Cyberattack,Malware,4077,Business,Consumable,2018,358,False,False
3,Unauthorized Access,NaN,590,Business,Manufacturing,2018,509,True,False
4,Cyberattack,Malware,956,Business,Fitness,2018,828,True,True
...,...,...,...,...,...,...,...,...,...
985,Cyberattack,Other,5886,Health,NaN,2024,0,False,False
986,Cyberattack,Other,2319,Business,Software,2024,387,True,False
987,Cyberattack,Ransomware,805,Business,Clothing,2024,19,True,True
988,Cyberattack,Other,978,Business,Other,2024,2,False,False


## Functions for exploring df and tests:
***

### Functions:

### Tests:

## Brainstorming
***

### Hypothesis Tests:

#### Industry Types

Government vs. Health

H0: Government mean affected <= Health mean affected

H1: Government mean affected > Health mean affected

#### Cyberattack Types

Phishing vs. Malware

H0: Phishing mean Affected <= Malware mean affected

H1: Phishing mean Affected > Malware mean affected

### Linear Regressions: